In [1]:
from google.colab import drive
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np


Connect to Google Drive

In [2]:
drive.mount('/content/drive')
# navigate to current directory
!pwd
%cd drive/MyDrive/CS439/
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive
/content
/content/drive/MyDrive/CS439


Load the training and test data here

In [3]:

class BLEDataset(Dataset):
    def __init__(self, input_data):
        # (x rows, 61 columns)
        self.features = input_data[:, :61]
        self.labels = F.one_hot(input_data[:, 61].long(), 25).float()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


Implement your neural network

In [4]:
class LOC_NN(nn.Module):
    def __init__(self, *args, **kwargs):
        super(LOC_NN, self).__init__()
        
        # self.rnn_s = list()
        # for i in range(12):
        #     self.rnn_s.append(nn.RNN(1, 5, batch_first=True).to(device))

        self.ll_s = []
        for i in range(12):
            self.ll_s.append(nn.Linear(5, 5).to(device))


        self.orien_em = nn.Embedding(4,10).to(device)
        self.hidden_layer = nn.Linear(70, 25).to(device)
        self.output_layer = nn.Linear(25, 25).to(device)

    def forward(self, x): # x:(batch_size, 61)
        batch_size, _ = x.shape
        rnn_vals_flatten = x[:, :60]
        rnn_vals_mat = torch.reshape(rnn_vals_flatten, (batch_size, 12, 5))
        orient = x[:,60]
        # # RNN input tensor (5,1), output (1, 5)
        # h_n0 = self.rnn_s[0](torch.reshape(rnn_vals_mat[:,0], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n1 = self.rnn_s[1](torch.reshape(rnn_vals_mat[:,1], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n2 = self.rnn_s[2](torch.reshape(rnn_vals_mat[:,2], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n3 = self.rnn_s[3](torch.reshape(rnn_vals_mat[:,3], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n4 = self.rnn_s[4](torch.reshape(rnn_vals_mat[:,4], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n5 = self.rnn_s[5](torch.reshape(rnn_vals_mat[:,5], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n6 = self.rnn_s[6](torch.reshape(rnn_vals_mat[:,6], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n7 = self.rnn_s[7](torch.reshape(rnn_vals_mat[:,7], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n8 = self.rnn_s[8](torch.reshape(rnn_vals_mat[:,8], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n9 = self.rnn_s[9](torch.reshape(rnn_vals_mat[:,9], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n10 = self.rnn_s[10](torch.reshape(rnn_vals_mat[:,10], (batch_size, 5, 1)).float())[1].squeeze(0)
        # h_n11 = self.rnn_s[11](torch.reshape(rnn_vals_mat[:,11], (batch_size, 5, 1)).float())[1].squeeze(0)

        ll_n0 = F.relu(self.ll_s[0](rnn_vals_mat[:,0].float()))
        ll_n1 = F.relu(self.ll_s[1](rnn_vals_mat[:,1].float()))
        ll_n2 = F.relu(self.ll_s[2](rnn_vals_mat[:,2].float()))
        ll_n3 = F.relu(self.ll_s[3](rnn_vals_mat[:,3].float()))
        ll_n4 = F.relu(self.ll_s[4](rnn_vals_mat[:,4].float()))
        ll_n5 = F.relu(self.ll_s[5](rnn_vals_mat[:,5].float()))
        ll_n6 = F.relu(self.ll_s[6](rnn_vals_mat[:,6].float()))
        ll_n7 = F.relu(self.ll_s[7](rnn_vals_mat[:,7].float()))
        ll_n8 = F.relu(self.ll_s[8](rnn_vals_mat[:,8].float()))
        ll_n9 = F.relu(self.ll_s[9](rnn_vals_mat[:,9].float()))
        ll_n10 = F.relu(self.ll_s[10](rnn_vals_mat[:,10].float()))
        ll_n11 = F.relu(self.ll_s[11](rnn_vals_mat[:,11].float()))

        # orientation embedding
        embed_output = self.orien_em(orient.int())
        # send concatenate data to input hidden layer
        # hidden_input = torch.cat((h_n0, h_n1, h_n2, h_n3, h_n4, h_n5, h_n6, h_n7, h_n8, h_n9, h_n10, h_n11, embed_output), dim=1)
        hidden_input = torch.cat((ll_n0, ll_n1, ll_n2, ll_n3, ll_n4, ll_n5, ll_n6, ll_n7, ll_n8, ll_n9, ll_n10, ll_n11, embed_output), dim=1)
        # hidden_input = F.dropout(hidden_input, p = 0.1)
        hidden_output = F.relu(self.hidden_layer(hidden_input))
        # hidden_output = F.dropout(hidden_output, p=0.2)
        class_output = self.output_layer(hidden_output)
        softmax_output = F.softmax(class_output, dim=1)
        return softmax_output
        

Set up dataset and loader

In [5]:
full_dataset = torch.tensor(pd.read_csv('data_set.csv', header=None).to_numpy())
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_data = torch.zeros((train_size, full_dataset.shape[1]))
test_data = torch.zeros((train_size, full_dataset.shape[1]))

# need to do split training dataset slices
slice_size = (full_dataset.shape[0]/100)
for i in range(100):
    full_offset = int(i * slice_size)
    train_offset = int(0.8 * i * slice_size)
    test_offset = int(0.2 * i * slice_size)
    train_data[train_offset:train_offset+8000] = full_dataset[full_offset:full_offset+8000]
    test_data[test_offset:test_offset+2000] = full_dataset[full_offset+8000:full_offset+10000]

batch_size = 100
train_dataset = BLEDataset(train_data)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_dataset = BLEDataset(test_data)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=True)

Train your network

In [6]:
def accuracy(output, labels, num_correct, num_total):
    # print("output shape:", output.shape)
    # print("labels shape:", labels.shape)
    _, predictions = torch.max(output, 1)
    _, actuals = torch.max(labels, 1)
    # print("predictions shape:", predictions.shape)
    # print("actuals shape:", actuals.shape)
    # compare predictions to true label
    correct = np.squeeze(predictions.eq(actuals.view_as(predictions)))
    # print("predict labels:", predictions)
    # print("actual labels:", actuals)
    # print("correct results:", correct)
    for i in range(len(actuals)):
        num_correct += correct[i].item()
        num_total += 1
    return num_correct, num_total

def train(model, data_loader, batch_size):  # try RNN next
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0003)
    criterion = nn.MSELoss()
    epochs = 50
    model = model.to(device)

    def train_epoch():
        num_correct = 0
        num_total = 0
        epoch_loss = 0
        model.train()
        for features, labels in data_loader:
            # run model and get output 
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
            # get accuracy 
            num_correct, num_total = accuracy(outputs, labels, num_correct, num_total)
            # get residuals
            loss = criterion(outputs, labels)
            # clear out gradient
            optimizer.zero_grad()
            # back propagate with loss funciton
            loss.backward()
            # gradient step
            optimizer.step()
            epoch_loss += loss.item()
        return num_correct/num_total, epoch_loss
        pass

    for e in range(epochs):
        print("epoch:", e)
        train_accuracy, train_loss = train_epoch()
        print("train accuracy:", train_accuracy, "train loss:", train_loss)

model = LOC_NN()
train(model, train_dataloader, batch_size)

epoch: 0
train accuracy: 0.35706 train loss: 244.7741823643446
epoch: 1
train accuracy: 0.70573 train loss: 156.3016298338771
epoch: 2
train accuracy: 0.80386 train loss: 109.39090877724811
epoch: 3
train accuracy: 0.85871125 train loss: 83.05453492701054
epoch: 4
train accuracy: 0.87768375 train loss: 68.07236839644611
epoch: 5
train accuracy: 0.8840725 train loss: 59.90869466739241
epoch: 6
train accuracy: 0.901885 train loss: 50.33133940771222
epoch: 7
train accuracy: 0.90502875 train loss: 46.24447065836284
epoch: 8
train accuracy: 0.906085 train loss: 44.08743194071576
epoch: 9
train accuracy: 0.906875 train loss: 42.67794895358384
epoch: 10
train accuracy: 0.90744375 train loss: 41.69471454073209
epoch: 11
train accuracy: 0.90779125 train loss: 40.98216472467175
epoch: 12
train accuracy: 0.90810625 train loss: 40.427024846489076
epoch: 13
train accuracy: 0.90836125 train loss: 39.9756665789173
epoch: 14
train accuracy: 0.90851875 train loss: 39.619331859459635
epoch: 15
train acc

Test your network

In [7]:
def evaluate(model):
    model.eval()

def test(model, test_loader):
    num_correct = 0
    num_total = 0
    test_loss = 0.0
    criterion = nn.MSELoss()
    evaluate(model)
    for features, labels in test_loader:
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features)
        # get accuracy 
        num_correct, num_total = accuracy(outputs, labels, num_correct, num_total)
        # add loss
        loss = criterion(outputs, labels.long())
        test_loss += loss.item() * features.size(0)
    return num_correct/num_total, test_loss

test_accuracy, test_loss = test(model, test_dataloader)
print("test accuracy:", test_accuracy, "test loss:", test_loss)

test accuracy: 0.22742625 test loss: 41775.97622126341
